In [3]:
from pprint import pprint
    
# Wine Quality Sample
def train():
    
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    import mlflow
    import mlflow.sklearn
    
    from urllib.parse import urlparse
    
    from joblib import dump, load
    
    import logging
    
    def logging():
        logging.basicConfig(level=logging.WARN)
        logger = logging.getLogger(__name__)
        return logger

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    pipe = Pipeline([("scaler", StandardScaler()), ("lr", LinearRegression())])
    
    with mlflow.start_run():
        model = pipe.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = model.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry 
        mlflow.sklearn.log_model(model, "model", serialization_format = "pickle")
            
if __name__ == "__main__":
    train()

  RMSE: 0.6603811224461618
  MAE: 0.5073644199888923
  R2: 0.3820931219110165
